In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [3]:
import wandb
run_id = wandb.util.generate_id(); run_id

'y1c8ru34'

In [4]:
wandb.init(id='19fc9tj5', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14037.137

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [8]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 23.4 s, sys: 812 ms, total: 24.3 s
Wall time: 24.3 s


In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [10]:
set_lr(3e-4)

In [11]:
model_stem = "11.29"
START_E = 16

In [ ]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 138 else 3e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 16

{'lat_losses/trn_control_loss': 0.11677384, 'lat_losses/trn_headings_loss': 0.00642764, 'lat_losses/trn_curvatures_loss': 0.05608058, 'lon_losses/trn_approaching_stop_loss': 0.03510941, 'lon_losses/trn_stop_dist_loss': 0.00326232, 'lon_losses/trn_stopped_loss': 0.00024409, 'lon_losses/trn_has_lead_loss': 0.0731411, 'lon_losses/trn_lead_dist_loss': 0.02052155, 'consistency losses/trn_steer_cost': 0.03522015, 'aux losses/trn_pitch_loss': 1.20456696, 'aux losses/trn_yaw_loss': 0.67029953, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 2282.87851477, 'consistency losses/trn_torque_delta_loss': 24.14666506, 'logistical/obs_consumed_per_second': 179.13385827, 'logistical/obs_generated_per_second': 112.27380952, 'logistical/slowest_runner_obs_per_sec': 10.9047619, 'logistical/data_consumption_ratio': 3.44814538, 'logistical/manual_train_pause': 0.05761905, 'logistical/max_param': 16.21623802, 'logistical/lr': 0.0003}
{'logist

{'logistical/obs_consumed_per_second': 174.921875, 'logistical/obs_generated_per_second': 174.0625, 'logistical/slowest_runner_obs_per_sec': 11.4609375, 'logistical/data_consumption_ratio': 1.00442497, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.08620465, 'lat_losses/trn_headings_loss': 0.00455115, 'lat_losses/trn_curvatures_loss': 0.02064374, 'lon_losses/trn_approaching_stop_loss': 0.0283566, 'lon_losses/trn_stop_dist_loss': 0.0041426, 'lon_losses/trn_stopped_loss': 0.00024404, 'lon_losses/trn_has_lead_loss': 0.08434606, 'lon_losses/trn_lead_dist_loss': 0.0303672, 'consistency losses/trn_steer_cost': 0.02481255, 'aux losses/trn_pitch_loss': 1.30118942, 'aux losses/trn_yaw_loss': 0.60066795, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.921875, 'consistency losses/trn_torque_loss': 2397.62559891, 'consistency losses/trn_torque_delta_loss': 26.16997541, 'logistical/max_param': 16.22237587, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_seco

{'logistical/obs_consumed_per_second': 176.7421875, 'logistical/obs_generated_per_second': 155.8203125, 'logistical/slowest_runner_obs_per_sec': 8.4296875, 'logistical/data_consumption_ratio': 1.14599036, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.14455354, 'lat_losses/trn_headings_loss': 0.00583775, 'lat_losses/trn_curvatures_loss': 0.0248743, 'lon_losses/trn_approaching_stop_loss': 0.04007345, 'lon_losses/trn_stop_dist_loss': 0.00377703, 'lon_losses/trn_stopped_loss': 0.00021696, 'lon_losses/trn_has_lead_loss': 0.06288382, 'lon_losses/trn_lead_dist_loss': 0.02425393, 'consistency losses/trn_steer_cost': 0.03403148, 'aux losses/trn_pitch_loss': 1.19636536, 'aux losses/trn_yaw_loss': 0.63403416, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 2385.52987289, 'consistency losses/trn_torque_delta_loss': 23.49112325, 'logistical/max_param': 16.22922707, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_pe

{'logistical/obs_consumed_per_second': 175.328125, 'logistical/obs_generated_per_second': 166.296875, 'logistical/slowest_runner_obs_per_sec': 11.71875, 'logistical/data_consumption_ratio': 1.05565447, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.07561755, 'lat_losses/trn_headings_loss': 0.00460332, 'lat_losses/trn_curvatures_loss': 0.02028407, 'lon_losses/trn_approaching_stop_loss': 0.03308146, 'lon_losses/trn_stop_dist_loss': 0.00374107, 'lon_losses/trn_stopped_loss': 5.424e-05, 'lon_losses/trn_has_lead_loss': 0.06872156, 'lon_losses/trn_lead_dist_loss': 0.02040624, 'consistency losses/trn_steer_cost': 0.01655084, 'aux losses/trn_pitch_loss': 1.53748322, 'aux losses/trn_yaw_loss': 0.55862427, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9140625, 'consistency losses/trn_torque_loss': 2251.21523881, 'consistency losses/trn_torque_delta_loss': 22.14017296, 'logistical/max_param': 16.23010254, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_s

{'logistical/obs_consumed_per_second': 173.3359375, 'logistical/obs_generated_per_second': 168.234375, 'logistical/slowest_runner_obs_per_sec': 10.6796875, 'logistical/data_consumption_ratio': 1.03180755, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.05950367, 'lat_losses/trn_headings_loss': 0.00383362, 'lat_losses/trn_curvatures_loss': 0.02318551, 'lon_losses/trn_approaching_stop_loss': 0.02754832, 'lon_losses/trn_stop_dist_loss': 0.00162978, 'lon_losses/trn_stopped_loss': 0.00016272, 'lon_losses/trn_has_lead_loss': 0.04473737, 'lon_losses/trn_lead_dist_loss': 0.02143874, 'consistency losses/trn_steer_cost': 0.01482552, 'aux losses/trn_pitch_loss': 1.17602921, 'aux losses/trn_yaw_loss': 0.59748459, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9375, 'consistency losses/trn_torque_loss': 2253.44217014, 'consistency losses/trn_torque_delta_loss': 18.35432827, 'logistical/max_param': 16.22716904, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_

...

{'logistical/obs_consumed_per_second': 171.15625, 'logistical/obs_generated_per_second': 165.4921875, 'logistical/slowest_runner_obs_per_sec': 10.9375, 'logistical/data_consumption_ratio': 1.03695395, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.05006433, 'lat_losses/trn_headings_loss': 0.00327859, 'lat_losses/trn_curvatures_loss': 0.01528592, 'lon_losses/trn_approaching_stop_loss': 0.01911599, 'lon_losses/trn_stop_dist_loss': 0.00194652, 'lon_losses/trn_stopped_loss': 2.712e-05, 'lon_losses/trn_has_lead_loss': 0.05430126, 'lon_losses/trn_lead_dist_loss': 0.0163663, 'consistency losses/trn_steer_cost': 0.01823625, 'aux losses/trn_pitch_loss': 1.2739563, 'aux losses/trn_yaw_loss': 0.41886139, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 2242.56364489, 'consistency losses/trn_torque_delta_loss': 23.46585624, 'logistical/max_param': 16.20120621, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_seco

{'logistical/obs_consumed_per_second': 169.3046875, 'logistical/obs_generated_per_second': 171.5, 'logistical/slowest_runner_obs_per_sec': 10.484375, 'logistical/data_consumption_ratio': 0.98945905, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.06365228, 'lat_losses/trn_headings_loss': 0.00361835, 'lat_losses/trn_curvatures_loss': 0.0155929, 'lon_losses/trn_approaching_stop_loss': 0.03570224, 'lon_losses/trn_stop_dist_loss': 0.00662482, 'lon_losses/trn_stopped_loss': 0.0003253, 'lon_losses/trn_has_lead_loss': 0.03803377, 'lon_losses/trn_lead_dist_loss': 0.01917654, 'consistency losses/trn_steer_cost': 0.02795732, 'aux losses/trn_pitch_loss': 1.25250244, 'aux losses/trn_yaw_loss': 0.56153202, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 2465.87007427, 'consistency losses/trn_torque_delta_loss': 24.37994667, 'logistical/max_param': 16.20868492, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_secon

{'logistical/obs_consumed_per_second': 171.390625, 'logistical/obs_generated_per_second': 166.71875, 'logistical/slowest_runner_obs_per_sec': 10.8828125, 'logistical/data_consumption_ratio': 1.02834741, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.1042515, 'lat_losses/trn_headings_loss': 0.006026, 'lat_losses/trn_curvatures_loss': 0.03461158, 'lon_losses/trn_approaching_stop_loss': 0.03107306, 'lon_losses/trn_stop_dist_loss': 0.00332442, 'lon_losses/trn_stopped_loss': 0.0001356, 'lon_losses/trn_has_lead_loss': 0.05108798, 'lon_losses/trn_lead_dist_loss': 0.01988576, 'consistency losses/trn_steer_cost': 0.03397217, 'aux losses/trn_pitch_loss': 1.39276123, 'aux losses/trn_yaw_loss': 0.54816246, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 2342.43311834, 'consistency losses/trn_torque_delta_loss': 24.91296956, 'logistical/max_param': 16.21473122, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_sec

{'logistical/obs_consumed_per_second': 173.6875, 'logistical/obs_generated_per_second': 173.5, 'logistical/slowest_runner_obs_per_sec': 11.5703125, 'logistical/data_consumption_ratio': 1.0020422, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.07678378, 'lat_losses/trn_headings_loss': 0.00437414, 'lat_losses/trn_curvatures_loss': 0.02104503, 'lon_losses/trn_approaching_stop_loss': 0.03958428, 'lon_losses/trn_stop_dist_loss': 0.05271364, 'lon_losses/trn_stopped_loss': 0.00016267, 'lon_losses/trn_has_lead_loss': 0.03385404, 'lon_losses/trn_lead_dist_loss': 0.01621806, 'consistency losses/trn_steer_cost': 0.02657819, 'aux losses/trn_pitch_loss': 1.14282227, 'aux losses/trn_yaw_loss': 0.64215183, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 2392.68951893, 'consistency losses/trn_torque_delta_loss': 25.14748846, 'logistical/max_param': 16.21916008, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second

{'logistical/obs_consumed_per_second': 177.0625, 'logistical/obs_generated_per_second': 170.5859375, 'logistical/slowest_runner_obs_per_sec': 12.2265625, 'logistical/data_consumption_ratio': 1.03869092, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.09136665, 'lat_losses/trn_headings_loss': 0.00410451, 'lat_losses/trn_curvatures_loss': 0.01873776, 'lon_losses/trn_approaching_stop_loss': 0.02689682, 'lon_losses/trn_stop_dist_loss': 0.00491791, 'lon_losses/trn_stopped_loss': 0.0001085, 'lon_losses/trn_has_lead_loss': 0.06495816, 'lon_losses/trn_lead_dist_loss': 0.02156831, 'consistency losses/trn_steer_cost': 0.02464268, 'aux losses/trn_pitch_loss': 1.22992325, 'aux losses/trn_yaw_loss': 0.71279526, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.953125, 'consistency losses/trn_torque_loss': 2319.91595697, 'consistency losses/trn_torque_delta_loss': 26.02016379, 'logistical/max_param': 16.24202728, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_s

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m.torch")

Error in callback <function _WandbInit._resume_backend at 0x7fb3b40e8ee0> (for pre_run_cell):


[autoreload of constants failed: Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 839, in exec_module
  File "<frozen importlib._bootstrap_external>", line 976, in get_code
  File "<frozen importlib._bootstrap_external>", line 906, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_r

Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7fb388ffb700> (for post_run_cell):


Exception: The wandb backend process has shutdown

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 167, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 109, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 401, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 232

In [13]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
